In [1]:
import sys
from pathlib import Path

SRC = Path().resolve() / "src"
if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

%load_ext autoreload
%autoreload 2

In [2]:
from data_utils import read_splits, truncate

# from data_utils import process_dataset
# process_dataset("data/raw_dataset.csv")

train_df, val_df, test_df = read_splits("data/dataset_processed.csv")

truncate_ratio = 1.0 # 0.05

train_texts = truncate(list(train_df['text']), truncate_ratio)
val_texsts = truncate(list(val_df['text']), truncate_ratio)

print(len(train_texts))

1280000


In [3]:
from transformers import BertTokenizerFast
from torch.utils.data import DataLoader
from next_token_dataset import NextTokenDataset

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

seq_len = 16
stride = 1

train_dataset = NextTokenDataset(train_texts, tokenizer, seq_len=seq_len, stride=stride)
val_dataset = NextTokenDataset(val_texsts, tokenizer, seq_len=seq_len, stride=stride)

print(f"Train samples count: {len(train_dataset)}")
print(f"Val samples count: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

100%|██████████| 160000/160000 [00:05<00:00, 27652.66it/s]

Train samples count: 4158366
Val samples count: 517103


In [ ]:
from lstm_model import NextTokenLSTM
from lstm_train import train_next_token
from device_utils import get_device

import torch
import torch.nn as nn

device = get_device()
print("Using device:", device)

model = NextTokenLSTM(
    vocab_size=tokenizer.vocab_size,
    emb_dim=256,
    hidden_dim=512,
    num_layers=2,
    pad_idx=tokenizer.pad_token_id or 0, # remove
    dropout_p=0.3,
)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

model = train_next_token(
    model,
    optimizer=optimizer,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=5
)

Using device: mps
